In [98]:
# Import Libraries and Dependencies
import pandas as pd

### 1. Combine and Clean the Data
#### Import CSVs

In [99]:
# Read the CSV files into DataFrames.
df = pd.read_csv('Resources/athletic_sales_2020.csv')

# Merge the two DataFrames together based on the Dates they share)
merge_df = pd.read_csv('Resources/athletic_sales_2021.csv') 

In [100]:
# Display the 2020 sales DataFrame
total_sold_2020 = df['total_sales'].sum()
print(f"Total sold in 2020: {total_sold_2020}")


Total sold in 2020: 181364121


In [101]:
# Display the 2021 sales DataFrame
total_sold_2021 = merge_df['total_sales'].sum()
print(f"Total sold in 2021: {total_sold_2021}")

Total sold in 2021: 717821450


In [102]:
# Check the 2020 sales data types.
df.dtypes
print(df.dtypes)


retailer             object
retailer_id           int64
invoice_date         object
region               object
state                object
city                 object
product              object
price_per_unit        int64
units_sold            int64
total_sales           int64
operating_profit    float64
sales_method         object
dtype: object


In [103]:
# Check the 2021 sales data types.
print(merge_df.dtypes)


retailer             object
retailer_id           int64
invoice_date         object
region               object
state                object
city                 object
product              object
price_per_unit        int64
units_sold            int64
total_sales           int64
operating_profit    float64
sales_method         object
dtype: object


#### Combine the sales data by rows.

In [104]:
# Combine the 2020 and 2021 sales DataFrames on the rows and reset the index.
combined_df = pd.concat([df,merge_df])
combined_df.reset_index(drop=True,inplace=True)


In [105]:
# Check if any values are null.
combined_df.isnull().sum()


retailer            0
retailer_id         0
invoice_date        0
region              0
state               0
city                0
product             0
price_per_unit      0
units_sold          0
total_sales         0
operating_profit    0
sales_method        0
dtype: int64

In [106]:
# Check the data type of each column
print(combined_df.dtypes)


retailer             object
retailer_id           int64
invoice_date         object
region               object
state                object
city                 object
product              object
price_per_unit        int64
units_sold            int64
total_sales           int64
operating_profit    float64
sales_method         object
dtype: object


In [107]:
combined_df.head()

,retailer,retailer_id,invoice_date,region,state,city,product,price_per_unit,units_sold,total_sales,operating_profit,sales_method
0,Foot Locker,1185732,1/1/20,Northeast,New York,New York,Men's Street Footwear,50,1200,600000,300000.00,In-store
1,Foot Locker,1185732,1/1/20,Northeast,Pennsylvania,Philadelphia,Women's Apparel,68,83,5644,2426.92,Online
2,Foot Locker,1185732,1/1/20,Northeast,Pennsylvania,Philadelphia,Women's Apparel,75,275,206250,61875.00,Outlet
3,Foot Locker,1185732,1/1/20,Northeast,New York,New York,Men's Street Footwear,34,384,13056,6789.12,Outlet
4,Foot Locker,1185732,1/1/20,Northeast,Pennsylvania,Philadelphia,Women's Apparel,53,83,4399,1407.68,Outlet


In [108]:
# Convert the "invoice_date" to a datetime datatype
combined_df["invoice_date"] = pd.to_datetime(combined_df["invoice_date"],format="%m/%d/%y")


In [109]:
# Confirm that the "invoice_date" data type has been changed.
print(combined_df.dtypes)

retailer                    object
retailer_id                  int64
invoice_date        datetime64[ns]
region                      object
state                       object
city                        object
product                     object
price_per_unit               int64
units_sold                   int64
total_sales                  int64
operating_profit           float64
sales_method                object
dtype: object


### 2. Determine which Region Sold the Most Products

#### Using `groupby`

In [110]:
# Show the number products sold for region, state, and city.
# Rename the sum to "Total_Products_Sold".
sold_most_products_gb = combined_df.groupby(["region","state","city"]).agg({"units_sold":"sum"})
sold_most_products_gb.columns = ["Total_Products_Sold"]




# Show the top 5 results.
(sold_most_products_gb.sort_values(by="Total_Products_Sold",ascending=False).head(5))
                            

Total_Products_Sold
region    state      city                              
Northeast New York   New York                    111954
South     Texas      Houston                      90322
West      California San Francisco                85478
                     Los Angeles                  76384
Southeast Florida    Miami                        73135

#### Using `pivot_table`

In [111]:
# Show the number products sold for region, state, and city.
sold_most_products_pt = pd.pivot_table(data=combined_df,index=["region","state","city"],aggfunc={"units_sold":"sum"})

# Rename the "units_sold" column to "Total_Products_Sold"
sold_most_products_pt.columns = ["Total_Products_Sold"]

# Show the top 5 results.
sold_most_products_pt.sort_values(by="Total_Products_Sold",ascending=False).head(5)

Total_Products_Sold
region    state      city                              
Northeast New York   New York                    111954
South     Texas      Houston                      90322
West      California San Francisco                85478
                     Los Angeles                  76384
Southeast Florida    Miami                        73135

### 3. Determine which Region had the Most Sales

#### Using `groupby`

In [112]:
# Show the total sales for the products sold for each region, state, and city.
# Rename the "total_sales" column to "Total Sales"
most_sales_region_gb = combined_df.groupby(["region","state","city"])[["total_sales"]].sum()
most_sales_region_gb.rename(columns={"total_sales":"Total Sales"},inplace=True)

# Show the top 5 results.
most_sales_region_gb.sort_values(by="Total Sales",ascending=False).head(5)

Total Sales
region    state          city                      
Northeast New York       New York          39801235
West      California     San Francisco     33973228
Southeast Florida        Miami             31600863
          South Carolina Charleston        29285637
          Florida        Orlando           27682851

#### Using `pivot_table`

In [113]:
# Show the total sales for the products sold for each region, state, and city.
most_sales_region_pt = pd.pivot_table(data=combined_df,index=["region","state","city"],aggfunc={"total_sales":"sum"})

# Optional: Rename the "total_sales" column to "Total Sales"
most_sales_region_pt.rename(columns= {"total_sales":"Total Sales"},inplace=True)

# Show the top 5 results.
most_sales_region_pt.sort_values(by="Total Sales",ascending=False).head(5)


Total Sales
region    state          city                      
Northeast New York       New York          39801235
West      California     San Francisco     33973228
Southeast Florida        Miami             31600863
          South Carolina Charleston        29285637
          Florida        Orlando           27682851

### 4. Determine which Retailer had the Most Sales

#### Using `groupby`

In [114]:
# Show the total sales for the products sold for each retailer, region, state, and city.
# Rename the "total_sales" column to "Total Sales"
retailer_most_sales_gb = combined_df.groupby(["retailer","region","state","city"]).agg({"total_sales":"sum"})
retailer_most_sales_gb.columns = ["Total Sales"]

# Show the top 5 results.
retailer_most_sales_gb.sort_values(by="Total Sales",ascending=False).head(5)


,,,,Total Sales
retailer,region,state,city,
West Gear,West,California,San Francisco,32794405
Kohl's,West,California,Los Angeles,25127160
Foot Locker,Northeast,New York,New York,25008568
West Gear,West,Washington,Seattle,24862675
Foot Locker,Southeast,South Carolina,Charleston,24822280


#### Using `pivot_table`

In [115]:
# Show the total sales for the products sold for each retailer, region, state, and city.
retailer_most_sales_pt = pd.pivot_table(data=combined_df,index=["retailer","region","state","city"],aggfunc={"total_sales":"sum"})

# Optional: Rename the "total_sales" column to "Total Sales"
retailer_most_sales_pt.rename(columns={"total_sales":"Total Sales"},inplace=True)

# Show the top 5 results.
retailer_most_sales_pt.sort_values(by="Total Sales",ascending=False).head(5)

,,,,Total Sales
retailer,region,state,city,
West Gear,West,California,San Francisco,32794405
Kohl's,West,California,Los Angeles,25127160
Foot Locker,Northeast,New York,New York,25008568
West Gear,West,Washington,Seattle,24862675
Foot Locker,Southeast,South Carolina,Charleston,24822280


### 5. Determine which Retailer Sold the Most Women's Athletic Footwear

In [116]:
combined_df.head()

,retailer,retailer_id,invoice_date,region,state,city,product,price_per_unit,units_sold,total_sales,operating_profit,sales_method
0,Foot Locker,1185732,2020-01-01,Northeast,New York,New York,Men's Street Footwear,50,1200,600000,300000.00,In-store
1,Foot Locker,1185732,2020-01-01,Northeast,Pennsylvania,Philadelphia,Women's Apparel,68,83,5644,2426.92,Online
2,Foot Locker,1185732,2020-01-01,Northeast,Pennsylvania,Philadelphia,Women's Apparel,75,275,206250,61875.00,Outlet
3,Foot Locker,1185732,2020-01-01,Northeast,New York,New York,Men's Street Footwear,34,384,13056,6789.12,Outlet
4,Foot Locker,1185732,2020-01-01,Northeast,Pennsylvania,Philadelphia,Women's Apparel,53,83,4399,1407.68,Outlet


In [117]:
# Filter the sales data to get the women's athletic footwear sales data.
women_sales_data = combined_df.loc[combined_df["product"].str.contains("Women's Athletic Footwear")]

#### Using `groupby`

In [118]:
# Show the total number of women's athletic footwear sold for each retailer, region, state, and city.
# Rename the "units_sold" column to "Womens_Footwear_Units_Sold"
women_apparel_gb = women_sales_data.groupby(["retailer","region","state","city"]).agg({"units_sold":"sum"})
women_apparel_gb.rename(columns={"units_sold":"Womens_Footwear_Units_Sold"},inplace=True)
# Show the top 5 results.
women_apparel_gb.sort_values(by="Womens_Footwear_Units_Sold",ascending=False).head(5)


,,,,Womens_Footwear_Units_Sold
retailer,region,state,city,
West Gear,West,California,San Francisco,12107
Foot Locker,Northeast,New York,New York,10996
Kohl's,West,California,Los Angeles,10826
Foot Locker,Southeast,South Carolina,Charleston,8814
Sports Direct,South,Texas,Dallas,8790


#### Using `pivot_table`

In [119]:
# Show the total number of women's athletic footwear sold for each retailer, region, state, and city.
women_apparel_pt = pd.pivot_table(data=women_sales_data,index=["retailer","region","state","city"],aggfunc={"units_sold":"sum"})

# Rename the "units_sold" column to "Womens_Footwear_Units_Sold"
women_apparel_pt.rename(columns={"units_sold":"Womens_Footwear_Units_Sold"},inplace=True)
# Show the top 5 results.
women_apparel_pt.sort_values(by="Womens_Footwear_Units_Sold",ascending=False).head(5)

,,,,Womens_Footwear_Units_Sold
retailer,region,state,city,
West Gear,West,California,San Francisco,12107
Foot Locker,Northeast,New York,New York,10996
Kohl's,West,California,Los Angeles,10826
Foot Locker,Southeast,South Carolina,Charleston,8814
Sports Direct,South,Texas,Dallas,8790


### 5. Determine the Day with the Most Women's Athletic Footwear Sales

In [120]:
# Create a pivot table with the 'invoice_date' column is the index, and the "total_sales" as the values.
women_pt = pd.pivot_table(data=women_sales_data,index=["invoice_date"],values=["total_sales"],aggfunc={"total_sales":"sum"})

# Optional: Rename the "total_sales" column to "Total Sales"
women_pt.rename(columns={"total_sales":"Total Sales"},inplace=True)

# Show the table.
women_pt

,Total Sales
invoice_date,
2020-01-04,402743
2020-01-05,141801
2020-01-11,129556
2020-01-17,173013
2020-01-22,388250
...,...
2021-12-22,200406
2021-12-23,808022
2021-12-24,190885


In [121]:
# Resample the pivot table into daily bins, and get the total sales for each day.
women_pt_resampled = women_pt.resample("D").sum()

# Sort the resampled pivot table in ascending order on "Total Sales".
women_pt_resampled.sort_values(by="Total Sales",ascending=False).head(10)


,Total Sales
invoice_date,
2021-07-16,1521825
2021-12-16,1473497
2021-06-17,1376988
2021-08-17,1086294
2021-07-23,1021806
2021-11-17,1021145
2021-12-09,915011
2021-06-24,884238
2021-07-09,869054


### 6.  Determine the Week with the Most Women's Athletic Footwear Sales

In [122]:
# Resample the pivot table into weekly bins, and get the total sales for each week.
women_pt_resampled_weekly = women_pt.resample("W").sum()

# Sort the resampled pivot table in ascending order on "Total Sales".
women_pt_resampled_weekly.sort_values(by="Total Sales",ascending=False).head(10)


,Total Sales
invoice_date,
2021-12-19,3098970
2021-12-12,2922161
2021-07-11,2835078
2021-07-18,2801449
2021-11-14,2531721
2021-08-22,2491259
2021-08-15,2463941
2021-11-21,2449537
2021-05-16,2422132
